# Movielens 영화 추천
- 별점을 시청횟수로 가정 
- 3점 미만은 선호하지 않는다고 가정

In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies['title']

0                         Toy Story (1995)
1                           Jumanji (1995)
2                  Grumpier Old Men (1995)
3                 Waiting to Exhale (1995)
4       Father of the Bride Part II (1995)
                       ...                
3878               Meet the Parents (2000)
3879            Requiem for a Dream (2000)
3880                      Tigerland (2000)
3881               Two Family House (2000)
3882                 Contender, The (2000)
Name: title, Length: 3883, dtype: object

# 2.분석하기
- ratings에 있는 유니크한 영화 갯수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기수)

In [5]:
print(ratings['movie_id'].nunique())
print(ratings['user_id'].nunique())
movies_count = ratings.groupby('movie_id')['user_id'].count()
# movies_count.sort_values(ascending=False).head(30)
a = movies_count.sort_values(ascending=False).head(30)
a = list(a.keys())
a

3628
6039


[2858,
 260,
 1196,
 1210,
 2028,
 589,
 593,
 1198,
 1270,
 2571,
 480,
 2762,
 608,
 110,
 1580,
 527,
 1197,
 2396,
 1617,
 318,
 858,
 1265,
 1097,
 2997,
 2716,
 296,
 356,
 1240,
 1,
 457]

# 3.선호하는 영화 5가지 골라 추가하기 

In [ ]:
my_lovemv = ['toy story','jumanji']

# 4.CSR matrix 직접만들기

In [ ]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

# 5. als_model = AlternatinngLeastSquares 모델 구성하여 훈련

# 6.내가 선호하는 5가지 영화 중 하나와 그 외의 영화를 하나를 골라 훈련된 모델이 예측한 선호도 파악

# 7. 내가 좋아하는 영황와 비슷한 영화를 추천받기

# 8. 내가 가장 좋아할 만한 영화들 추천받기